In [5]:
import lidar
import rosbag
from pathlib import Path
from dask.distributed import Client
from dask import delayed
import dask.dataframe as dd
import dask

client = Client()  # start distributed scheduler locally.  Launch dashboard
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

/opt/conda/lib/python3.7/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 43205 instead
  http_address["port"], self.http_server.port
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
bagfile = "/workspaces/lidar/tests/testdata/big.bag"

In [7]:
%%time
rosbag.Bag(bagfile)

CPU times: user 12.3 s, sys: 3.77 s, total: 16.1 s
Wall time: 13.3 s


In [8]:
%%time
test0= lidar.Dataset.from_file(Path(bagfile), start_frame_number = 0, end_frame_number = 25)

[0 1 2 3 4 5 6 7 8]
100%|██████████| 9/9 [00:03<00:00,  2.81it/s]
[ 9 10 11 12 13 14 15 16]
100%|██████████| 8/8 [00:02<00:00,  2.72it/s]
[17 18 19 20 21 22 23 24]
100%|██████████| 8/8 [00:02<00:00,  2.98it/s]CPU times: user 20.6 s, sys: 3.01 s, total: 23.6 s
Wall time: 20.5 s



In [9]:
len(test0)

25

In [14]:
test1 = lidar.Dataset.from_file(Path(bagfile), start_frame_number = 11, end_frame_number = 20)

100%|██████████| 9/9 [00:05<00:00,  1.58it/s]


In [15]:
len(test0)

10

In [16]:
len(test1)

9

In [8]:
import numpy as np
max_size = 10
length = 25
test = np.arange(length)

result = np.array_split(test, (len(test) // max_size)+1)

In [9]:
result

[array([0, 1, 2, 3, 4, 5, 6, 7, 8]),
 array([ 9, 10, 11, 12, 13, 14, 15, 16]),
 array([17, 18, 19, 20, 21, 22, 23, 24])]

In [10]:
len(result)

3

In [6]:
chunk_num = 0
results = []
for chunk in result:
    print("new")
    print(chunk[0])
    print(chunk[-1])
    dataset = lidar.Dataset.from_file(Path(bagfile), start_frame_number = chunk[0], end_frame_number = chunk[-1]+1)
    print(len(chunk))
    print(dataset)
    results.append(dataset)


NameError: name 'result' is not defined

In [7]:
data = []
[data.extend(dataset.data) for dataset in results]
timestamps = []
[timestamps.extend(dataset.timestamps) for dataset in results]
meta = dataset.meta

NameError: name 'dataset' is not defined

In [34]:
joined = lidar.Dataset(data,timestamps,meta)

In [35]:
len(joined)

25

In [36]:
%%time
test25 = lidar.Dataset.from_file(Path(bagfile), start_frame_number = 0, end_frame_number = 25)

  0%|          | 0/25 [00:00<?, ?it/s]
0
100%|██████████| 25/25 [00:14<00:00,  1.73it/s]CPU times: user 34 s, sys: 7.38 s, total: 41.4 s
Wall time: 33.5 s



In [37]:
len(test25)

25

In [38]:
joined.timestamps == test25.timestamps

True

In [39]:
res = joined.data[20].compute() == test25.data[20].compute()

In [40]:
res.describe()

,x,y,z,intensity,t,reflectivity,ring,noise,range,original_id
count,112257,112257,112257,112257,112257,112257,112257,112257,112257,112257
unique,1,1,1,1,1,1,1,1,1,1
top,True,True,True,True,True,True,True,True,True,True
freq,112257,112257,112257,112257,112257,112257,112257,112257,112257,112257


Ok, so this works, now what about parallel

In [78]:
%%time
chunk_num = 0
results = []
for chunk in result:
    dataset = delayed(lidar.Dataset.from_file)(Path(bagfile), start_frame_number = chunk[0], end_frame_number = chunk[-1])
    results.append(dataset)


CPU times: user 7.67 ms, sys: 4.31 ms, total: 12 ms
Wall time: 9.25 ms


In [72]:
%%time
res = dd.compute(*results)

CPU times: user 1.41 s, sys: 411 ms, total: 1.82 s
Wall time: 15.2 s


In [2]:
t="Dataset([Delayed('DataFrame-a1a9aa96-0827-4cfc-ab3c-0cc2a5bd2f5f'), Delayed('DataFrame-4e4ac984-4651-45f6-876f-2dcb48501c35')],[datetime.datetime(2020, 6, 22, 13, 40, 42, 657267), datetime.datetime(2020, 6, 22, 13, 40, 42, 755912)],{'orig_file': '/workspaces/lidar/tests/testdata/test.bag', 'topic': '/os1_cloud_node/points'})"

In [3]:
len(t)

326